<a href="https://colab.research.google.com/github/alevalespejo/Nutritional_Epidemiology_lab/blob/main/MBA_Bokk_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/MinaBeric/Market_Basket_Analysis/blob/main/Market_basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spark Context creation**

In [1]:
# Install Java (required for Spark)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark 3.5.2 via pip
!pip install pyspark==3.5.2
!pip install -q findspark unidecode

# Set environment variables (Java is needed)
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Initialize findspark
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# Verify installation
print("PySpark Version:", spark.version)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812369 sha256=04fec7ff6a1a98f554a9c1c8e621115656173869d52b80d70cfa64699f2ae0b9
  Stored in directory: /root/.cache/pip/wheels/9d/29/ee/3a756632ca3f0a6870933bac1c9db6e4af2c068f019aba0ee1
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00
PySpark Version: 3.5.2


In [2]:
import pyspark
print(pyspark.__version__)


3.5.2


In [3]:
import pyspark
sc=spark.sparkContext
from pyspark.sql import functions as F
from pyspark.sql.functions import col, desc
from pyspark.sql.functions import count, desc
from pyspark.sql.functions import collect_list, explode, lower, regexp_replace, trim, when
from pyspark.sql.types import StringType
import re

# **Import Dataset from Kaggle**

In [4]:
import os
os.environ['KAGGLE_USERNAME'] = "alessandraespejo"
os.environ['KAGGLE_KEY'] = "bdbc4a269fd587b09cf04e7963b942c9"
!kaggle datasets download -d "mohamedbakhet/amazon-books-reviews"

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0


In [5]:
!unzip /content/amazon-books-reviews.zip -d data

Archive:  /content/amazon-books-reviews.zip
  inflating: data/Books_rating.csv   
  inflating: data/books_data.csv     


In [6]:
!rm /content/data/books_data.csv
#!rm /content/amazon-books-reviews.zip

In [7]:
df = spark.read.options(header=True).csv('/content/data/Books_rating.csv')
df.show(10, truncate=False)

+----------+------------------------------+-----+--------------+--------------------------------------+------------------+------------+-----------+------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **Data pre processing and EDA**

In [ ]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [8]:
colummns_to_keep = ["Id", "Title", "User_id","review/text"]
df_w = df.select(*colummns_to_keep)

In [9]:
df_w = df_w.na.drop()  #drop empty, NA cells
df_w = df_w.dropDuplicates() # drop repeated cells

In [ ]:
df_w.describe()

summary,Id,Title,User_id,review/text
count,2414744,2414744,2414744,2414744
mean,1.0712653625246209E9,2029.3904406561594,18.349261744966444,9.964967334178803E8
stddev,1.29198629286454E9,1745.2422304244342,22.096134847767384,4.2197291598831904E8
min,0001047604,""""""" Film technique",""""" Film acting """"""",""""" """"House of Ma..."
max,B0064P287I,xBase Programming...,AZZZZW74AAX75,~~~~~~~~~~~~~~~~~...


In [ ]:
# 1. Group by title and count occurrences
title_counts = df_w.groupBy("Title").count()

# 2. Order by count in descending order
top_titles = title_counts.orderBy(desc("count"))

# 3. Show the top N titles (e.g., top 10)
top_titles.show(10, truncate=False)  # Show top 10, avoid truncating titles

+--------------------------+-----+
|Title                     |count|
+--------------------------+-----+
|The Hobbit                |17621|
|Pride and Prejudice       |17500|
|Atlas Shrugged            |10616|
|Wuthering Heights         |8490 |
|Great Expectations        |5612 |
|Mere Christianity         |5453 |
|The Picture of Dorian Gray|5208 |
|Brave New World           |5004 |
|Persuasion                |4939 |
|Of Mice and Men           |4503 |
+--------------------------+-----+
only showing top 10 rows



# Now let's see how many books do we have

In [ ]:
unici_titles_count = df_w.select("Title").distinct().count()
print(f"Number of unique titles: {unici_titles_count}")

Number of unique titles: 206633


In [ ]:
df_groupe = df_w.groupBy("Title").agg(count("*").alias("count"))
#df_groupe.show(10, truncate=False)

we want to keep only meaninful books

In [ ]:
# Calculate percentiles of review counts
percentiles = df_groupe.approxQuantile("count", [0.25, 0.5, 0.75], 0.01)
print(f"25th percentile: {percentiles[0]}")
print(f"50th percentile (median): {percentiles[1]}")
print(f"75th percentile: {percentiles[2]}")

25th percentile: 1.0
50th percentile (median): 2.0
75th percentile: 6.0


In [10]:
# Instead of the 75th percentile (6 reviews) 60th percentile (4/5 reviews):
df_filtered = df_groupe.withColumn("filter_book",F.when(F.col("count") >= 6, "kept").otherwise("removed"))


NameError: name 'df_groupe' is not defined

In [ ]:
nuovo_df = df_w.join(df_filtered, on=["Title"], how="left")
nuovo_df = nuovo_df.filter(F.col("filter_book") == "kept")

In [ ]:
nuovo_df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- count: long (nullable = true)
 |-- filter_book: string (nullable = true)



let's look at how many books we have after this initial choice

In [ ]:
conti_titoli = nuovo_df.select("Title").distinct().count()
print(f"New number of unique titles: {conti_titoli}")

New number of unique titles: 68541


In [ ]:
# 1. Group by title and count occurrences
title_count = nuovo_df.groupBy("Title").count()

# 2. Order by count in descending order
top_titles = title_count.orderBy(desc("count"))

# 3. Show the top N titles (e.g., top 10)
top_titles.show(10, truncate=False)

+--------------------------+-----+
|Title                     |count|
+--------------------------+-----+
|The Hobbit                |17621|
|Pride and Prejudice       |17500|
|Atlas Shrugged            |10616|
|Wuthering Heights         |8490 |
|Great Expectations        |5612 |
|Mere Christianity         |5453 |
|The Picture of Dorian Gray|5208 |
|Brave New World           |5004 |
|Persuasion                |4939 |
|Of Mice and Men           |4503 |
+--------------------------+-----+
only showing top 10 rows



In [ ]:
nuovo_df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- review/text: string (nullable = true)
 |-- count: long (nullable = true)
 |-- filter_book: string (nullable = true)



# **We know there might be some problems with the column "titles" due to different editions,typos and so on. Let's see how to fix this**

First let's group by titles and normalize them

In [ ]:
df_norm = nuovo_df.groupBy("Title").agg(count("*").alias("conteggio_recensioni"))

In [ ]:
remove_words = ["trilogy", "volume", "edition", "special", "unabridged", "audiobook", "cd", "audio",
                "illustrated", "updated", "movie tie-in", "ebook", "hardcover", "slipcover","narrated", "edition",
                "revised"]

df_norm = (df_norm
    .withColumn("cleaned_title", lower(col("Title")))

    .withColumn("cleaned_title", regexp_replace(col("cleaned_title"), "\\p{M}", ""))

    .withColumn("cleaned_title", regexp_replace(col("cleaned_title"), "[^a-zA-Z0-9'\\s]", ""))

    .withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\b(" + "|".join(remove_words) + r")\b", ""))

    .withColumn("cleaned_title", trim(regexp_replace(col("cleaned_title"), r"\s+", " ")))

    .withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\b(\w+)\s+\1\b", r"\1"))

    .withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"&", "and"))

)

#df_norm = df_norm.withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\(\s*book\s*\d+\s*\)", ""))
#df_norm = df_norm.withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\b(and|&)\b", "and"))
# df_norm = df_norm.withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\btho\b", "the"))
#df_norm = df_norm.withColumn("cleaned_title", regexp_replace(col("cleaned_title"), r"\bteh\b", "the"))
#df_norm = df_norm.withColumn("cleaned_title", trim(regexp_replace(col("cleaned_title"), r"\s+", " ")))  # Ensure single spaces


when analysing the data i found that tmany of of the most known books title wwere creating problems

In [ ]:

hobbit_pattern = r"(?i)\b(hobitt?|there\s+and\s+back\s+again)\b"

# Add final normalization step
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(hobbit_pattern),
        F.lit("the hobbit")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
gatsby_pattern = r"(?i)\b(gatsby|great gatsby|leading english literature library|everyman|signet|revised)\b"

df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(gatsby_pattern),
        F.lit("the great gatsby")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
anna_karenina_pattern = r"(?i)\b(anna karenina|translated from the russian by constance garnett|bt fritz eichberg)\b"

df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(anna_karenina_pattern),
        F.lit("anna karenina")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
# Heart of Darkness Pattern
heart_of_darkness_pattern = r"(?i)\b(heart\s+of\s+darkness|heritage\s+club\s+series)\b"
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(heart_of_darkness_pattern),
        F.lit("heart of darkness")
    ).otherwise(F.col("cleaned_title"))
)


In [ ]:
# Moby Dick Pattern
moby_dick_pattern = r"(?i)\b(moby\s+dick|or\s+the\s+whale|the\s+world's\s+classics|mobydick|or\s+the\s+white\s+whale|world's\s+greatest\s+literature\s+17|harper's\s+modern\s+classics)\b"
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(moby_dick_pattern),
        F.lit("moby dick")
    ).otherwise(F.col("cleaned_title"))
)


In [ ]:
# No Highway Pattern
no_highway_pattern = r"(?i)\b(no\s+highway|a\s+novel|charnwood\s+library)\b"
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(no_highway_pattern),
        F.lit("no highway")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
lord_of_the_rings_pattern = r"(?i)\b(lord\s+of\s+the\s+rings|three\s+volumes\s+in\s+slipcase|3\s+set|three\s+boxed\s+set|the\s+fellowship\s+of\s+the\s+ring|the\s+return\s+of\s+the\s+king|the\s+two\s+towers|boxed\s+set)\b"

# Add normalization step for "The Lord of the Rings"
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(lord_of_the_rings_pattern),
        F.lit("the lord of the rings")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
monte_cristo_pattern = r"(?i)\b(count\s+of\s+monte\s+cristo|rittenhouse\s+classics|abridged|golden\s+picture\s+classic|novels\s+of\s+alexandre\s+dumas|monte\s+cristo)\b"
df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(monte_cristo_pattern),
        F.lit("the count of monte cristo")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
mockingbird_pattern = r"(?i)\b(to\s+kill\s+a\s+mocking(?:bird|bird)|harper\s+lee's\s+to\s+kill\s+a\s+mockingbird\s+barron's\s+book\s+notes)\b"

df_norm = df_norm.withColumn(
    "cleaned_title",
    F.when(
        F.col("cleaned_title").rlike(mockingbird_pattern),
        F.lit("to kill a mockingbird")
    ).otherwise(F.col("cleaned_title"))
)

In [ ]:
df_norm = df_norm.na.drop()

In [ ]:
df_norm.printSchema()

root
 |-- Title: string (nullable = true)
 |-- conteggio_recensioni: long (nullable = false)
 |-- cleaned_title: string (nullable = true)



**the number of unique cleaned_title should be less than "title" :**  **68541**

In [ ]:
normalized_count = df_norm.select("cleaned_title").distinct().count()
print(f"Number of cleaned_title: {normalized_count}")

Number of cleaned_title: 66046


before 56736

In [ ]:
df_norm.select("Title", "cleaned_title").show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|Title                                                                                                                            |cleaned_title                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|"""A"" IS FOR ALIBI"                                                                                                             |a is for alibi                                                                                                   |
|"""Beatles"" Il

In [ ]:
from pyspark.sql.functions import col

hobbit_df = df_norm.filter(col("cleaned_title").contains('hobbit'))

In [ ]:
hobbit_df.show(truncate=False)

+----------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------+
|Title                                                                                         |count|cleaned_title                                                                            |
+----------------------------------------------------------------------------------------------+-----+-----------------------------------------------------------------------------------------+
|The Hobbit                                                                                    |17621|the hobbit                                                                               |
|The Hobbit; Or, There and Back Again                                                          |3451 |the hobbit                                                                               |
|The Hobbit There and Back Again   

In [ ]:
cleaned_df = nuovo_df.join(df_norm, on=["Title"], how="left")

In [ ]:
cleaned_df = cleaned_df.na.drop()

In [ ]:
tenere = ["Id", "cleaned_title", "User_id","review/text"]
cleaned_df = cleaned_df.select(*tenere)

Now I check for those users who rewied the same book more than once

In [ ]:
multiples_reviews_df = cleaned_df.groupBy("User_id", "cleaned_title") \
                        .agg(count("*").alias("conteggio")) \
                        .filter(col("conteggio") > 1)

distinct_users_df = multiples_reviews_df.select("User_id").distinct()
user_count = distinct_users_df.count()
print(f"Number of users who reviewed the same book multiple times: {user_count}")

Number of users who reviewed the same book multiple times: 139617


To solve the root problem I eliminate those user who review the same book mutliple times an keep only one of their multiples reviews

In [ ]:
cleaned_df = cleaned_df.dropDuplicates(["User_id", "cleaned_title"])

I want to keep for my analysis only those users who are considered as active (at least 3 reviews on distinct books )

In [ ]:
utente_counts = cleaned_df.groupBy("User_id").agg(count("*").alias("totale_rec"))
cleaned_df = cleaned_df.join(utente_counts, "User_id").filter(col("totale_rec") >= 3)

In [ ]:
cleaned_df.count()

872884

# **CREATE** **BASKETS**

In [ ]:
baskets_df = cleaned_df.groupBy("User_ID") \
    .agg(collect_list("cleaned_title").alias("book_title")) \
    .select("book_title")

In [ ]:
baskets_df.count()

115158

In [ ]:
baskets_df.select(F.size("book_title").alias("basket_size")) \
      .groupBy("basket_size").count().orderBy("basket_size").show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

# **Sampling**

In [ ]:
sample_fraction = 0.01
sampled_df = baskets_df.sample(
    withReplacement = False,
    fraction = sample_fraction,
    seed=14).cache()

In [ ]:
sampled_df.count()

In [ ]:
sampled_df.show(10, truncate=False)

# RDD Baskets creation

In [ ]:
rdd_baskets = sampled_df.select("book_title").rdd.map(lambda row: row[0])

rdd_baskets.take(5)

In [ ]:
rdd_baskets.getNumPartitions()

200

In [ ]:
rdd_baskets.first()

['why i am not a christian',
 'why i am not a christian and other essays on religion and related subjects',
 'veni vidi vici conquer your enemies impress your friends with everyday latin']

In [ ]:
lens = rdd_baskets.map(lambda x: len(x))

In [ ]:
max_len = lens.max()
max_len

582

In [ ]:
mean_len = lens.mean()
mean_len

4.663607483233323

In [ ]:
min_len = lens.min()
min_len

2

# Creation of the Hash table

In [ ]:
all_books = rdd_baskets.flatMap(lambda books: books).distinct()


In [ ]:
all_books.first()

In [ ]:
check_tot_books = all_books.count()
check_tot_books

In [ ]:
book_to_index = all_books.zipWithIndex().collectAsMap()


In [ ]:
for key, value in list(book_to_index.items())[:10]:
    print(f'book: {key}, Index: {value}')

In [ ]:
rdd_r3 = rdd_baskets.map(lambda books: {book_to_index[book] for book in books if book in book_to_index})


In [ ]:
rdd_r3.first()

#**A PRIORI ALGORITHM**

**Support threshold**

In [ ]:
support_threshold = round(rdd_baskets.count() * 0.015)

In [ ]:
support_threshold

325

In [ ]:
support_percentage = (support_threshold /rdd_baskets.count()) * 100

In [ ]:
support_percentage

1.5001764913519238

In [ ]:
from itertools import combinations
import matplotlib.pyplot as plt

In [ ]:
def apriori(baskets_rdd, threshold, book_hash_table):
    itemset_counts = {}

    # Pass 1: Find Frequent Singletons
    first_pass = (baskets_rdd.flatMap(lambda basket: [(book, 1) for book in basket])
                  .reduceByKey(lambda a, b: a + b)
                  .filter(lambda item_count: item_count[1] > threshold)
    )

    first_count = first_pass.count()
    max_singleton = first_pass.max(lambda x: x[1])
    most_frequent_book = list(book_hash_table.keys())[list(book_hash_table.values()).index(max_singleton[0])]

    print("Number of Frequent Singletons:", first_count)
    print("Most Frequent Singleton:", most_frequent_book)

    frequents = set(first_pass.map(lambda x: (x[0],)).collect())
    itemset_counts["Size 1"] = first_count

    k = 2

    while True:
        candidate_itemsets = (baskets_rdd.flatMap(lambda basket: [(comb, 1) for comb in combinations(sorted(basket), k)
                                       if all(subset in frequents for subset in combinations(comb, k-1))])
                              .reduceByKey(lambda x, y: x + y)
                              .filter(lambda x: x[1] > threshold)
        )

        candidates_counts = candidate_itemsets.count()

        if candidates_counts != 0:
            print(f"Number of frequent itemsets of size {k} are", candidates_counts)

            max_itemsets = candidate_itemsets.max(lambda x: x[1])
            max_items = [list(book_hash_table.keys())[list(book_hash_table.values()).index(book)] for book in max_itemsets[0]]
            print(f"The Most frequent itemset of size {k} is", max_items)

            itemset_counts[f"Size {k}"] = candidates_counts
            frequents = set(candidate_itemsets.map(lambda x: x[0]).collect())
            k += 1
        else:
            print(f"There are no more frequent itemsets of size {k}")
            break

    # Plot frequent itemsets count
    plt.figure(figsize=(10, 6))
    plt.bar(itemset_counts.keys(), itemset_counts.values(), color='pink')
    plt.xlabel("Itemset Size")
    plt.ylabel("Count of Frequent Itemsets")
    plt.title("Frequent Itemsets Count by Size")
    plt.show()

# Run Apriori
apriori(rdd_r3, support_threshold, book_to_index)
